# Main Jupyter notebook for Urdf2Moon

Jupyter to add the gravity term to the optimal torque solution

In [1]:
import casadi as cs
from urdf2casadi import urdfparser as u2c
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class Urdf2Moon:
    def __init__(self, urdf_path, root, tip):
        self.robot_parser = self.load_urdf(urdf_path)

        # Store inputs
        self.root = root
        self.tip = tip
        
        # Get basic info
        self.num_joints = self.get_joints_n(self.root, self.tip)
        self.q, self.q_dot, self.u_hat = self.define_symbolic_vars(self.num_joints)
        self.M, self.Cq, self.G = self.get_motion_equation_matrix(self.root, self.tip, self.q, self.q_dot)
        self.upper_x, self.lower_x = self.get_limits(self.root, self.tip)
        self.fk_dict = self.robot_parser.get_forward_kinematics(self.root, self.tip)

    def define_symbolic_vars(self, num_joints):
        q = cs.SX.sym("q", num_joints)
        q_dot = cs.SX.sym("q_dot", num_joints)
        return q, q_dot, u_hat
    def load_urdf(self, urdf_path):
        robot_parser = u2c.URDFparser()
        robot_parser.from_file(urdf_path)
        return robot_parser
    def get_joints_n(self, root, tip):
        return self.robot_parser.get_n_joints(root, tip) #return the number of actuated joints
    def get_limits(self, root, tip):
        _, _, upper, lower = self.robot_parser.get_joint_info(root, tip)
        return upper, lower
    def get_motion_equation_matrix(self, root, tip, q, q_dot):
        # load inertia terms (function)
        M_sym = self.robot_parser.get_inertia_matrix_crba(root, tip)
        # load gravity terms (function)
        gravity_u2c = [0, 0, -9.81]
        G_sym = self.robot_parser.get_gravity_rnea(root, tip, gravity_u2c)
        # load Coriolis terms (function)
        C_sym = self.robot_parser.get_coriolis_rnea(root, tip)
        return M_sym, C_sym, G_sym

In [3]:
if __name__ == '__main__':
    urdf_path = "../urdf/rrbot.urdf"
    root = "link1" 
    end = "link4"

    model = Urdf2Moon(urdf_path, root, end)

In [4]:
# Load forward kinematics (4x4 matrix)
T_fk = model.fk_dict["T_fk"]

In [5]:
dim = model.num_joints
epsilon = cs.SX.sym("epsilon", dim) # create symbolic variables to perform derivatives in epsilon
epsilon_q = T_fk(model.q)[0:3:2,3]  # the same variable but q_dipendent

x = epsilon[0]
y = epsilon[1]

x0 = 0.2
y0 = 0.2
R = 0.1

In [6]:
C = (x - x0)**2 + (y - y0)**2 - R**2           # define C(epsilon)
C_func = cs.Function('C_func', [epsilon], [C], # create a funcion, useful to express C as function of q
                     ["epsilon"], ["C"])
C_q = C_func(epsilon_q)                        # C(q)

In [7]:
C_epsilon = cs.jacobian(C, epsilon)            # define dC/depsilon
C_epsilon_func = cs.Function('C_epsilon_func', [epsilon], [C_epsilon], # create the function
                             ["epsilon"], ["C_epsilon"])
C_epsilon_q = C_epsilon_func(epsilon_q)        # C_epsilon(q)

In [8]:
S = cs.SX.sym("S", dim)
S[0] = -C_epsilon[1]    # S is defined to have C.T*S=0
S[1] =  C_epsilon[0]
S_func = cs.Function('S_func', [epsilon], [S], # create the function
                     ["epsilon"], ["S"])
S_q = S_func(epsilon_q) # S(q)
print("check if 0 =", C_epsilon@S)

check if 0 = 0


In [9]:
J = cs.jacobian(T_fk(model.q)[0:3:2,3], model.q) # Jacobian, just select the position one (last column of matrix)

In [10]:
# parameters
ni = 0.5
alpha = 0.1
Kb = 1.0

In [11]:
gamma = -cs.pinv(J)@((C_epsilon_q@C_q.T*ni).T + S_q*alpha)
gamma_epsilon = cs.jacobian(gamma, model.q) # dgamma/dq
gamma_dot = gamma_epsilon@model.q_dot       # dgamma/dt = dgamma/dq * dq/dt

In [12]:
def evaluate_tau (model, C, C_epsilon, gamma, gamma_dot, J, ni, Kb):
    a = model.Cq(model.q, model.q_dot) + model.G(model.q)  
    p = (cs.pinv(model.q_dot-gamma)@(C.T*C_epsilon@(J@model.q_dot+C_epsilon.T*C*ni))).T
    b = gamma_dot - p - Kb*(model.q_dot-gamma)
    tau = a + model.M(model.q)@b
    return tau

In [13]:
tau = evaluate_tau (model, C_q, C_epsilon_q, gamma, gamma_dot, J, ni, Kb)
tau_func = cs.Function("tau_func", [model.q, model.q_dot], [tau], 
                       ["q", "q_dot"], ["tau"])

In [14]:
q = [0.5, 0.5]
q_dot = [1.0, 1.0]

In [15]:
tau_val = tau_func(q, q_dot)

In [16]:
tau_val

DM([133.918, 12.1951])